<a href="https://colab.research.google.com/github/RagaaM/Scriba/blob/main/dataset-testing-trial-1/ScribaTrial2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [53]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

IMAGE_SIZE = [224, 224]

trainSet = '/content/drive/MyDrive/programs/Scriba/dataset/train'
testSet = '/content/drive/MyDrive/programs/Scriba/dataset/test'

inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)


# don't train existing weights
for layer in inception.layers:
    layer.trainable = False

# useful for getting number of output classes
folders = glob('/content/drive/MyDrive/programs/Scriba/dataset/train/*')


# our layers - you can add more if you want
x = Flatten()(inception.output)

#if you have multiple outputs ---> use softmax
# if you have binary outputs ---> use sigmoid
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=inception.input, outputs=prediction)


# view the structure of the model
model.summary()


Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_1222 (Conv2D)            (None, 111, 111, 32) 864         input_14[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1222 (Batch (None, 111, 111, 32) 96          conv2d_1222[0][0]                
__________________________________________________________________________________________________
activation_1222 (Activation)    (None, 111, 111, 32) 0           batch_normalization_1222[0][0]   
____________________________________________________________________________________________

In [54]:

# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)


# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/programs/Scriba/dataset/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')


Found 45 images belonging to 3 classes.


In [55]:

test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/programs/Scriba/dataset/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 12 images belonging to 3 classes.


In [56]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
2/2 [==============================] - 12s 4s/step - loss: 2.4437 - accuracy: 0.2889 - val_loss: 6.0754 - val_accuracy: 0.5000
Epoch 2/20
2/2 [==============================] - 6s 4s/step - loss: 2.2022 - accuracy: 0.7111 - val_loss: 0.1003 - val_accuracy: 0.9167
Epoch 3/20
2/2 [==============================] - 6s 3s/step - loss: 0.1842 - accuracy: 0.9778 - val_loss: 1.5828 - val_accuracy: 0.7500
Epoch 4/20
2/2 [==============================] - 6s 3s/step - loss: 0.6325 - accuracy: 0.8889 - val_loss: 3.0473 - val_accuracy: 0.6667
Epoch 5/20
2/2 [==============================] - 6s 4s/step - loss: 0.7165 - accuracy: 0.9111 - val_loss: 0.7275 - val_accuracy: 0.8333
Epoch 6/20
2/2 [==============================] - 6s 4s/step - loss: 0.0515 - accuracy: 0.9778 - val_loss: 2.7195e-04 - val_accuracy: 1.0000
Epoch 7/20
2/2 [==============================] - 6s 4s/step - loss: 5.2982e-09 - accuracy: 1.0000 - val_loss: 2.0801e-05 - val_accuracy: 1.0000
Epoch 8/20
2/2 [============